In [2]:
import matplotlib.pyplot as plt 
import json 

In [ ]:
import matplotlib.pyplot as plt
import json
import numpy as np

# ====================================================================
# A. 模型家族和名称映射定义 (沿用 V7 修正)
# ====================================================================
model_family_map = {
    'qwen-plus-0728': 'Alibaba (Qwen)', 'qwen3-next-80b-a3b-thinking': 'Alibaba (Qwen)', 'qwen3-235b-a22b-instruct-2507': 'Alibaba (Qwen)', 'qwen3-next-80b-a3b-instruct': 'Alibaba (Qwen)', 'qwen3-max-0923': 'Alibaba (Qwen)', 
    'DeepSeek-V3.1-Terminus-nothinking': 'DeepSeek', 'DeepSeek-V3.2-Exp-nothinking': 'DeepSeek', 'DeepSeek-V3.1-Terminus-thinking': 'DeepSeek', 'DeepSeek-V3.2-Exp-thinking': 'DeepSeek', 
    'Claude-Opus-4.1-nothinking': 'Anthropic (Claude)', 'Claude-4-Sonnet-nothinking': 'Anthropic (Claude)', 'Claude-Sonnet-4.5-nothinking': 'Anthropic (Claude)', 'Claude-Sonnet-4.5-thinking': 'Anthropic (Claude)', 
    'Gemini-2.5-Pro': 'Google (Gemini)', 'Gemini-2.5-Flash-Lite': 'Google (Gemini)', 'Gemini-2.5-Flash': 'Google (Gemini)', 'Gemini-2.5-Flash.1': 'Google (Gemini)', 'Gemini-2.5-Flash-Lite-Preview-2509': 'Google (Gemini)', 'Gemini-2.5-Flash-Preview-2509': 'Google (Gemini)', 
    'GPT-5-chat': 'OpenAI (GPT)', 'GPT-5-high': 'OpenAI (GPT)', 'GPT-5-medium': 'OpenAI (GPT)', 'qwen3-235b-a22b-instruct': 'Alibaba (Qwen)', 'qwen3-235b-a22b-thinking': 'Alibaba (Qwen)',
    'Kimi-K2-0711': 'Moonshot (Kimi)', 'Kimi-K2-0905': 'Moonshot (Kimi)', 
    'GLM-4.5': 'ZhipuAI (GLM)', 'GLM-4.6': 'ZhipuAI (GLM)', "Gemini-3-pro": "Google (Gemini)", "GPT-5.1": "OpenAI (GPT)", "DeepSeek-V3.2-thinking": "DeepSeek","Gemini-3-Pro-Preview": "Google (Gemini)",
    'Claude-Opus-4.1-thinking': 'Anthropic (Claude)', "Kimi-K2-thinking":"Moonshot (Kimi)",
    # 'Minimax-M2': 'Minimax',
}
model_name_map = {
    'qwen-plus-0728': 'Qwen-Plus-0728', 'qwen3-next-80b-a3b-thinking': 'Qwen3-Next-80b-tk', 'qwen3-235b-a22b-instruct-2507': 'Qwen3-235b', 'qwen3-next-80b-a3b-instruct': 'Qwen3-Next-80b-I', 'qwen3-max-0923': 'Qwen3-Max',
    'DeepSeek-V3.1-Terminus-nothinking': 'DeepSeek-V3.1-ntk', 'DeepSeek-V3.2-Exp-nothinking': 'DeepSeek-V3.2-E-ntk', 'DeepSeek-V3.1-Terminus-thinking': 'DeepSeek-V3.1-T-tk', 'DeepSeek-V3.2-Exp-thinking': 'DeepSeek-V3.2-E-tk',
    'Claude-Opus-4.1-nothinking': 'Claude-Opus-4.1-ntk','Claude-Opus-4.1-thinking': 'Claude-Opus-4.1-tk', 'Claude-4-Sonnet-nothinking': 'Claude-4-Sonnet-ntk', 'Claude-Sonnet-4.5-nothinking': 'Claude-S4.5-ntk', 'Claude-Sonnet-4.5-thinking': 'Claude-S4.5-tk',
    'Gemini-2.5-Pro': 'Gemini-2.5-Pro', 'Gemini-2.5-Flash-Lite': 'Gemini-Flash-L', 'Gemini-2.5-Flash': 'Gemini-2.5-Flash', 'Gemini-2.5-Flash.1': 'Gemini-Flash.1', 'Gemini-2.5-Flash-Lite-Preview-2509': 'Gemini-Flash-L-P', 'Gemini-2.5-Flash-Preview-2509': 'Gemini-Flash-P',
    'GPT-5-chat': 'GPT-5-Chat', 'GPT-5-high': 'GPT-5-High', 'GPT-5-medium': 'GPT-5-Medium',
    'Kimi-K2-0711': 'Kimi-K2-0711', 'Kimi-K2-0905': 'Kimi-K2-0905',
    'GLM-4.5': 'GLM-4.5', 'GLM-4.6': 'GLM-4.6',
    # 'Minimax-M2': 'Minimax-M2', 
    "Gemini-3-pro":"Gemini-3-pro", "GPT-5.1":"GPT-5.1-high", "DeepSeek-V3.2-thinking":"DeepSeek-V3.2-tk",
    'qwen3-235b-a22b-instruct': 'Qwen3-235B-Instruct', 'qwen3-235b-a22b-thinking': 'Qwen3-235B-thinking', "Gemini-3-Pro-Preview":"Gemini-3-pro"
}
# 注意：这里Anthropic (Claude)的颜色值在您的代码中是 #044F9B，但为了保持V4-V9的颜色一致性，我使用之前确认的 #1E90FF，除非您明确要求更改。
family_colors_base = {
    'Alibaba (Qwen)': "#7F07B3", 'DeepSeek': "#0D7B0D", 'Anthropic (Claude)': '#1E90FF', 'Google (Gemini)': "#BE250A",        
    'OpenAI (GPT)': '#000000', 'Moonshot (Kimi)': "#B8E518", 'ZhipuAI (GLM)': '#87CEFA', 'Minimax': '#DAA520',                
}
# ====================================================================

# B. 数据载入和预处理 (T_k=2 降序排序)
try:
    with open('paper_overall.json', 'r') as f:
        data = json.load(f)
except FileNotFoundError:
    print("Error: swiss_high_results_paper.json not found. Please place the file in the same directory.")
    exit()

del data["Minimax-M2"]

all_model_names = list(data.keys())
families = sorted(list(set(model_family_map.values())))
    
T2_scores_map = {name: data.get(name, [0, 0, 0])[2] for name in all_model_names}
sorted_models = sorted(all_model_names, key=lambda name: T2_scores_map.get(name, 0), reverse=True) 

model_scores_data = []
for name in sorted_models:
    scores = data.get(name, [0, 0, 0])
    model_scores_data.append({
        'name': name,
        'simplified_name': model_name_map.get(name, name),
        'family': model_family_map.get(name, 'Other'),
        'scores': scores,
    })

# C. Matplotlib 配置
AXIS_LABEL_SIZE = 20
MODEL_LABEL_SIZE = 18
LEGEND_FONT_SIZE = 18

plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = AXIS_LABEL_SIZE
plt.rcParams['axes.labelweight'] = 'bold'

num_models = len(sorted_models)
fig_width = 20 
fig, ax = plt.subplots(figsize=(fig_width, 8), dpi=200) 

# D. 绘制垂直趋势线图
x_pos = np.arange(num_models)
X_JITTER = [-0.1, 0, 0.1] 
ELIMINATION_LABELS = ['No elimination ($T_k=0$)', 'Drop 1/round ($T_k=1$)', 'Drop 2/round ($T_k=2$)']

legend_handles = []
markers_drawn = set()

# 用于收集 T2 点的坐标
t2_x_coords = []
t2_y_coords = []

for i, model in enumerate(model_scores_data):
    if "minimax" in model["name"].lower():
        continue 
    base_x = x_pos[i]
    color = family_colors_base.get(model['family'], '#000000') 
    
    # 1. 绘制连接线 (趋势线)
    x_coords_line = [base_x + X_JITTER[j] for j in range(3)]
    y_coords_line = model['scores']
    
    ax.plot(x_coords_line, y_coords_line, 
            color=color, 
            linestyle='-', 
            linewidth=2.5, 
            alpha=0.7, 
            zorder=1)

    # 2. 绘制数据点 (T0, T1, T2)
    for j in range(3):
        x = base_x + X_JITTER[j]
        y = model['scores'][j]
        
        marker_color = 'black'
        marker_size = 10 
        
        if j == 0: 
            marker = 'o' 
            alpha = 0.5
        elif j == 1: 
            marker = 'v'
            alpha = 0.8
        else: # j == 2 (T_k=2)
            marker = 's'
            alpha = 1.0
            # 收集 T2 点坐标
            t2_x_coords.append(x)
            t2_y_coords.append(y)
        
        ax.plot(x, y, 
                marker=marker, markersize=marker_size, 
                linestyle='None', 
                color=color, 
                markeredgecolor=marker_color, 
                markeredgewidth=0.8, 
                alpha=alpha, 
                zorder=2)
        
        if (j, marker) not in markers_drawn:
            legend_handles.append(ax.plot([], [], marker=marker, markersize=marker_size, 
                                          linestyle='None', color='gray', label=ELIMINATION_LABELS[j])[0])
            markers_drawn.add((j, marker))

# F. 绘制 T_k=2 的水平虚线 (新增部分)
ax.plot(t2_x_coords, t2_y_coords, 
        color='gray', 
        linestyle='--', # 虚线
        linewidth=1.5, 
        alpha=0.8, 
        zorder=0) # 放在最底层

# E. 图表配置和美化 (沿用 V9 设置)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_linewidth(1.5) 
ax.spines['left'].set_linewidth(1.5) 

ax.set_xticks(x_pos)
ax.set_xticklabels([model['simplified_name'] for model in model_scores_data], 
                   rotation=45, ha='right', fontsize=MODEL_LABEL_SIZE) 

ax.set_ylabel("Expected Win Score ($E[S_m]$)", fontsize=AXIS_LABEL_SIZE)
ax.grid(True, which='major', linestyle='--', linewidth=0.5, alpha=0.4, axis='y', zorder=0)
ax.set_xlabel("", fontsize=AXIS_LABEL_SIZE) 

# 使用用户提供的 Y 轴范围
ax.set_ylim(-0.1, 10.0)

# 7. 绘制图例 (Legend) - 沿用 V9 修正的位置

# 创建家族图例 (颜色)
family_handles = []
for family in sorted(families):
    color = family_colors_base.get(family)
    handle = ax.plot([], [], marker='s', markersize=10, linestyle='-', linewidth=2.5, color=color, alpha=0.8, 
                     markeredgecolor='black', markeredgewidth=0.8, label=f'{family} Family')[0]
    family_handles.append(handle)

# 家族图例：设置为 4 列 (两行), 位置在 1.08
legend1 = ax.legend(handles=family_handles, labels=[h.get_label() for h in family_handles], 
                    loc='upper center', bbox_to_anchor=(0.5, 1.05), # 调整到 1.08
                    ncol=4, 
                    frameon=False, fontsize=LEGEND_FONT_SIZE,
                    columnspacing=1.5, handletextpad=0.5)

ax.add_artist(legend1)

# 阶段图例：保持单行, 位置在 0.95
legend2 = ax.legend(handles=legend_handles, labels=[h.get_label() for h in legend_handles],
                    loc='upper center', bbox_to_anchor=(0.5, 0.9), # 调整到 0.95
                    ncol=len(legend_handles), 
                    frameon=False, fontsize=LEGEND_FONT_SIZE,
                    columnspacing=1.5, handletextpad=0.5)

# 调整 top 区域，使其包含图例 (从 0.80 调整到 0.90)
plt.subplots_adjust(bottom=0.20, top=0.90, left=0.05, right=0.98) 

plt.savefig("swiss_sensitivity_original_sgpqa.png", dpi=200, bbox_inches='tight')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import json
import numpy as np
from matplotlib.patches import FancyArrowPatch

# ====================================================================
# A. 模型家族和名称映射定义 (沿用)
# ====================================================================
model_family_map = {
    'qwen-plus-0728': 'Alibaba (Qwen)', 'qwen3-next-80b-a3b-thinking': 'Alibaba (Qwen)', 'qwen3-235b-a22b-instruct-2507': 'Alibaba (Qwen)', 'qwen3-next-80b-a3b-instruct': 'Alibaba (Qwen)', 'qwen3-max-0923': 'Alibaba (Qwen)', 
    'DeepSeek-V3.1-Terminus-nothinking': 'DeepSeek', 'DeepSeek-V3.2-Exp-nothinking': 'DeepSeek', 'DeepSeek-V3.1-Terminus-thinking': 'DeepSeek', 'DeepSeek-V3.2-Exp-thinking': 'DeepSeek', 
    'Claude-Opus-4.1-nothinking': 'Anthropic (Claude)', 'Claude-4-Sonnet-nothinking': 'Anthropic (Claude)', 'Claude-Sonnet-4.5-nothinking': 'Anthropic (Claude)', 'Claude-Sonnet-4.5-thinking': 'Anthropic (Claude)', 
    'Gemini-2.5-Pro': 'Google (Gemini)', 'Gemini-2.5-Flash-Lite': 'Google (Gemini)', 'Gemini-2.5-Flash': 'Google (Gemini)', 'Gemini-2.5-Flash.1': 'Google (Gemini)', 'Gemini-2.5-Flash-Lite-Preview-2509': 'Google (Gemini)', 'Gemini-2.5-Flash-Preview-2509': 'Google (Gemini)', 
    'GPT-5-chat': 'OpenAI (GPT)', 'GPT-5-high': 'OpenAI (GPT)', 'GPT-5-medium': 'OpenAI (GPT)', 
    'Kimi-K2-0711': 'Moonshot (Kimi)', 'Kimi-K2-0905': 'Moonshot (Kimi)', 
    'GLM-4.5': 'ZhipuAI (GLM)', 'GLM-4.6': 'ZhipuAI (GLM)', "Gemini-3-pro": "Google (Gemini)", "GPT-5.1": "OpenAI (GPT)", "DeepSeek-V3.2-thinking": "DeepSeek",
    'Minimax-M2': 'Minimax',
}
model_name_map = {
    'qwen-plus-0728': 'Qwen-Plus-0728', 'qwen3-next-80b-a3b-thinking': 'Qwen3-Next-80b-tk', 'qwen3-235b-a22b-instruct-2507': 'Qwen3-235b', 'qwen3-next-80b-a3b-instruct': 'Qwen3-Next-80b-I', 'qwen3-max-0923': 'Qwen3-Max',
    'DeepSeek-V3.1-Terminus-nothinking': 'DeepSeek-V3.1-ntk', 'DeepSeek-V3.2-Exp-nothinking': 'DeepSeek-V3.2-E-ntk', 'DeepSeek-V3.1-Terminus-thinking': 'DeepSeek-V3.1-T-tk', 'DeepSeek-V3.2-Exp-thinking': 'DS-V3.2-E-tk',
    'Claude-Opus-4.1-nothinking': 'Claude-Opus-4.1', 'Claude-4-Sonnet-nothinking': 'Claude-4-Sonnet', 'Claude-Sonnet-4.5-nothinking': 'Claude-S4.5', 'Claude-Sonnet-4.5-thinking': 'Claude-S4.5-tk',
    'Gemini-2.5-Pro': 'Gemini-2.5-Pro', 'Gemini-2.5-Flash-Lite': 'Gemini-Flash-L', 'Gemini-2.5-Flash': 'Gemini-2.5-Flash', 'Gemini-2.5-Flash.1': 'Gemini-Flash.1', 'Gemini-2.5-Flash-Lite-Preview-2509': 'Gemini-Flash-L-P', 'Gemini-2.5-Flash-Preview-2509': 'Gemini-Flash-P',
    'GPT-5-chat': 'GPT-5-Chat', 'GPT-5-high': 'GPT-5-High', 'GPT-5-medium': 'GPT-5-Medium',
    'Kimi-K2-0711': 'Kimi-K2-0711', 'Kimi-K2-0905': 'Kimi-K2-0905',
    'GLM-4.5': 'GLM-4.5', 'GLM-4.6': 'GLM-4.6',
    'Minimax-M2': 'Minimax-M2', "Gemini-3-pro":"Gemini-3-pro", "GPT-5.1":"GPT-5.1-high", "DeepSeek-V3.2-thinking":"DeepSeek-V3.2-tk"
}
family_colors_base = {
    'Alibaba (Qwen)': "#7F07B3", 'DeepSeek': "#0D7B0D", 'Anthropic (Claude)': '#1E90FF', 'Google (Gemini)': "#BE250A",        
    'OpenAI (GPT)': '#000000', 'Moonshot (Kimi)': "#B8E518", 'ZhipuAI (GLM)': '#87CEFA', 'Minimax': '#DAA520',
}
# ====================================================================

# B. 数据载入和计算新指标
try:
    with open('swiss_high_results_new.json', 'r') as f:
        data = json.load(f)
except FileNotFoundError:
    print("Error: swiss_high_results_paper.json not found. Please place the file in the same directory.")
    exit()

if "Minimax-M2" in data:
    del data["Minimax-M2"]

processed_data = []
for name, scores in data.items():
    if name not in model_family_map:
        continue 
    
    score_t0 = scores[0] 
    drop_t2_t0 = scores[2] - scores[0] 
    
    processed_data.append({
        'name': name,
        'simplified_name': model_name_map.get(name, name),
        'family': model_family_map.get(name, 'Other'),
        'score_t0': score_t0,
        'drop': drop_t2_t0,
        'color': family_colors_base.get(model_family_map.get(name, 'Other'), '#808080')
    })

# C. Matplotlib 配置
AXIS_LABEL_SIZE = 18
LEGEND_FONT_SIZE = 12
MARKER_SIZE = 10
MODEL_NAME_FONT_SIZE = 9 
MUTATION_SCALE_VALUE = 1 # 箭头 scale 设置为 1

plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = AXIS_LABEL_SIZE
plt.rcParams['axes.labelweight'] = 'bold'

# D. 绘制散点图
fig_width = 14
fig, ax = plt.subplots(figsize=(fig_width, 8), dpi=300) 

# 手动调整标签位置
manual_offsets = {
    'Qwen3-Next-80b-tk': (-5, 10), 
    'DS-V3.2-E-ntk': (5, -15),
    'GLM-4.5': (-25, 5), 
    'Qwen-Plus-0728': (-35, 0),
    'Gemini-Flash-L': (5, -15), 
    'Claude-S4.5': (-5, 10), 
    'GPT-5-Medium': (-5, 10),
}

# 绘制数据点和标签
for model in processed_data:
    x = model['score_t0']
    y = model['drop']
    
    ax.scatter(x, y, 
               color=model['color'], 
               edgecolor='black', 
               linewidth=1, 
               marker='o', 
               s=MARKER_SIZE**2,
               alpha=0.8)
    
    offset = manual_offsets.get(model['simplified_name'], (5, 5))
    


# E. 坐标轴移动和样式化
xmin_data, xmax_data = ax.get_xlim()
ymin_data_actual, ymax_data_actual = ax.get_ylim()

# 新的 X 轴 Y 坐标 (Y 轴的最小值)
ORIGIN_Y_AXIS = ymin_data_actual - (ymax_data_actual - ymin_data_actual) * 0.05 # 稍微低于数据最低点
ORIGIN_X_AXIS = xmin_data # Y轴的 X 坐标

# 调整 X/Y 轴的显示范围
x_range = xmax_data - xmin_data
new_xmax = xmax_data + x_range * 0.15 
ax.set_xlim(xmin_data, new_xmax)
ax.set_ylim(ORIGIN_Y_AXIS, ymax_data_actual + (ymax_data_actual - ymin_data_actual) * 0.05)
xmax, ymin = ax.get_xlim()[1], ax.get_ylim()[0] # 更新最新的轴范围
ymax = ax.get_ylim()[1]


# 1. 隐藏多余的边框
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False) 
ax.spines['top'].set_visible(False) # 隐藏默认顶部边框
ax.spines['left'].set_visible(False) # 隐藏默认左侧边框


# 2. X 轴 (绘制在 ORIGIN_Y_AXIS 处)
# 绘制 X 轴线
ax.plot([ORIGIN_X_AXIS, xmax], [ORIGIN_Y_AXIS, ORIGIN_Y_AXIS], color='black', lw=1.5, clip_on=False)
# X轴刻度：放置在 X 轴 (ORIGIN_Y_AXIS) 上方
ax.xaxis.set_ticks_position('bottom') # 刻度线依然是bottom，但标签通过pad移到上方
ax.tick_params(axis='x', which='major', direction='in', length=6, width=1.5, labelsize=12, pad=5) # pad向上移

# 3. Y 轴 (绘制在 ORIGIN_X_AXIS 处)
# 绘制 Y 轴线
ax.plot([ORIGIN_X_AXIS, ORIGIN_X_AXIS], [ORIGIN_Y_AXIS, ymax], color='black', lw=1.5, clip_on=False)
# Y轴刻度：放置在 Y 轴 (ORIGIN_X_AXIS) 左侧
ax.yaxis.set_ticks_position('left')
ax.tick_params(axis='y', which='major', direction='in', length=6, width=1.5, labelsize=12, pad=5)


# F. 箭头 (确保可见且方向正确)
arrow_head_size = 10 # 箭头大小的基准
arrow_style = f'->, head_length={arrow_head_size}, head_width={arrow_head_size * 0.8}'

# X 轴箭头 (向右)
arrow_x = FancyArrowPatch((xmax, ORIGIN_Y_AXIS), (xmax + (xmax - ORIGIN_X_AXIS) * 0.01, ORIGIN_Y_AXIS), 
                          arrowstyle=arrow_style, 
                          color='black', lw=1.5, 
                          mutation_scale=MUTATION_SCALE_VALUE, # 设置为1
                          clip_on=False)
ax.add_patch(arrow_x)

# Y 轴箭头 (向上)
arrow_y = FancyArrowPatch((ORIGIN_X_AXIS, ymax), (ORIGIN_X_AXIS, ymax + (ymax - ORIGIN_Y_AXIS) * 0.01), 
                          arrowstyle=arrow_style, 
                          color='black', lw=1.5,
                          mutation_scale=MUTATION_SCALE_VALUE, # 设置为1
                          clip_on=False)
ax.add_patch(arrow_y)


# 零基准线 (如果 ORIGIN_Y_AXIS 不为 0，则 0 处仍然可以有虚线参考)
if ORIGIN_Y_AXIS != 0:
    ax.axhline(0, color='gray', linestyle=':', linewidth=0.5, zorder=0) 
ax.grid(True, which='major', linestyle=':', linewidth=0.5, alpha=0.4, zorder=0, axis='both') 

# X轴标签
ax.set_xlabel("Base Performance ($T_k=0$)", fontsize=AXIS_LABEL_SIZE, labelpad=15)
# Y轴标签 (标准旋转)
ax.set_ylabel("Performance Drop ($\Delta E[S_m]$)", 
              fontsize=AXIS_LABEL_SIZE, 
              rotation=90, 
              ha='center', 
              labelpad=15) 


# G. 图例 (Legend) - 放在最底部
families = sorted(list(set(model_family_map.values())))
family_handles = []
for family in families:
    color = family_colors_base.get(family)
    handle = ax.scatter([], [], marker='o', s=MARKER_SIZE**2, color=color, edgecolor='black', 
                        label=f'{family} Family')
    family_handles.append(handle)

# legend = ax.legend(handles=family_handles, 
#                    loc='lower center', 
#                    bbox_to_anchor=(0.5, -0.25), 
#                    ncol=3, 
#                    frameon=False, 
#                    fontsize=LEGEND_FONT_SIZE,
#                    title="Model Family",
#                    title_fontsize=LEGEND_FONT_SIZE)

# 调整布局和保存
plt.subplots_adjust(left=0.1, right=0.95, top=0.95, bottom=0.25) 
plt.savefig("robustness_vs_performance_scatter_clean_V16_Final.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import json
import numpy as np

# ====================================================================
# A. 模型家族和名称映射定义 (沿用 V7 修正)
# ====================================================================
model_family_map = {
    'qwen-plus-0728': 'Alibaba (Qwen)', 'qwen3-next-80b-a3b-thinking': 'Alibaba (Qwen)', 'qwen3-235b-a22b-instruct-2507': 'Alibaba (Qwen)', 'qwen3-next-80b-a3b-instruct': 'Alibaba (Qwen)', 'qwen3-max-0923': 'Alibaba (Qwen)', 
    'DeepSeek-V3.1-Terminus-nothinking': 'DeepSeek', 'DeepSeek-V3.2-Exp-nothinking': 'DeepSeek', 'DeepSeek-V3.1-Terminus-thinking': 'DeepSeek', 'DeepSeek-V3.2-Exp-thinking': 'DeepSeek', 
    'Claude-Opus-4.1-nothinking': 'Anthropic (Claude)', 'Claude-4-Sonnet-nothinking': 'Anthropic (Claude)', 'Claude-Sonnet-4.5-nothinking': 'Anthropic (Claude)', 'Claude-Sonnet-4.5-thinking': 'Anthropic (Claude)', 
    'Gemini-2.5-Pro': 'Google (Gemini)', 'Gemini-2.5-Flash-Lite': 'Google (Gemini)', 'Gemini-2.5-Flash': 'Google (Gemini)', 'Gemini-2.5-Flash.1': 'Google (Gemini)', 'Gemini-2.5-Flash-Lite-Preview-2509': 'Google (Gemini)', 'Gemini-2.5-Flash-Preview-2509': 'Google (Gemini)', 
    'GPT-5-chat': 'OpenAI (GPT)', 'GPT-5-high': 'OpenAI (GPT)', 'GPT-5-medium': 'OpenAI (GPT)', 
    'Kimi-K2-0711': 'Moonshot (Kimi)', 'Kimi-K2-0905': 'Moonshot (Kimi)', 
    'GLM-4.5': 'ZhipuAI (GLM)', 'GLM-4.6': 'ZhipuAI (GLM)', 
#     'Minimax-M2': 'Minimax',
}
model_name_map = {
    'qwen-plus-0728': 'Qwen-Plus-0728', 'qwen3-next-80b-a3b-thinking': 'Qwen3-Next-80b-tk', 'qwen3-235b-a22b-instruct-2507': 'Qwen3-235b', 'qwen3-next-80b-a3b-instruct': 'Qwen3-Next-80b-I', 'qwen3-max-0923': 'Qwen3-Max',
    'DeepSeek-V3.1-Terminus-nothinking': 'DS-V3.1-ntk', 'DeepSeek-V3.2-Exp-nothinking': 'DS-V3.2-E-ntk', 'DeepSeek-V3.1-Terminus-thinking': 'DS-V3.1-T-tk', 'DeepSeek-V3.2-Exp-thinking': 'DS-V3.2-E-tk',
    'Claude-Opus-4.1-nothinking': 'Claude-Opus-4.1', 'Claude-4-Sonnet-nothinking': 'Claude-4-Sonnet', 'Claude-Sonnet-4.5-nothinking': 'Claude-S4.5', 'Claude-Sonnet-4.5-thinking': 'Claude-S4.5-tk',
    'Gemini-2.5-Pro': 'Gemini-2.5-Pro', 'Gemini-2.5-Flash-Lite': 'Gemini-Flash-L', 'Gemini-2.5-Flash': 'Gemini-2.5-Flash', 'Gemini-2.5-Flash.1': 'Gemini-Flash.1', 'Gemini-2.5-Flash-Lite-Preview-2509': 'Gemini-Flash-L-P', 'Gemini-2.5-Flash-Preview-2509': 'Gemini-Flash-P',
    'GPT-5-chat': 'GPT-5-Chat', 'GPT-5-high': 'GPT-5-High', 'GPT-5-medium': 'GPT-5-Medium',
    'Kimi-K2-0711': 'Kimi-K2-0711', 'Kimi-K2-0905': 'Kimi-K2-0905',
    'GLM-4.5': 'GLM-4.5', 'GLM-4.6': 'GLM-4.6',
#     'Minimax-M2': 'Minimax-M2',
}
# 注意：这里Anthropic (Claude)的颜色值在您的代码中是 #044F9B，但为了保持V4-V9的颜色一致性，我使用之前确认的 #1E90FF，除非您明确要求更改。
family_colors_base = {
    'Alibaba (Qwen)': "#7F07B3", 'DeepSeek': "#0D7B0D", 'Anthropic (Claude)': '#1E90FF', 'Google (Gemini)': "#BE250A",        
    'OpenAI (GPT)': '#000000', 'Moonshot (Kimi)': "#B8E518", 'ZhipuAI (GLM)': '#87CEFA', 'Minimax': '#DAA520',                
}
# ====================================================================

# B. 数据载入和预处理 (T_k=2 降序排序)
try:
    with open('swiss_high_results_sensitivity_glm.json', 'r') as f:
        data = json.load(f)
except FileNotFoundError:
    print("Error: swiss_high_results_paper.json not found. Please place the file in the same directory.")
    exit()

del data["Minimax-M2"]

all_model_names = list(data.keys())
families = sorted(list(set(model_family_map.values())))
    
T2_scores_map = {name: data.get(name, [0, 0, 0])[2] for name in all_model_names}
sorted_models = sorted(all_model_names, key=lambda name: T2_scores_map.get(name, 0), reverse=True) 

model_scores_data = []
for name in sorted_models:
    scores = data.get(name, [0, 0, 0])
    model_scores_data.append({
        'name': name,
        'simplified_name': model_name_map.get(name, name),
        'family': model_family_map.get(name, 'Other'),
        'scores': scores,
    })

# C. Matplotlib 配置
AXIS_LABEL_SIZE = 20
MODEL_LABEL_SIZE = 18
LEGEND_FONT_SIZE = 18

plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = AXIS_LABEL_SIZE
plt.rcParams['axes.labelweight'] = 'bold'

num_models = len(sorted_models)
fig_width = 20 
fig, ax = plt.subplots(figsize=(fig_width, 8), dpi=200) 

# D. 绘制垂直趋势线图
x_pos = np.arange(num_models)
X_JITTER = [-0.1, 0, 0.1] 
ELIMINATION_LABELS = ['No elimination ($T_k=0$)', 'Drop 1/round ($T_k=1$)', 'Drop 2/round ($T_k=2$)']

legend_handles = []
markers_drawn = set()

# 用于收集 T2 点的坐标
t2_x_coords = []
t2_y_coords = []

for i, model in enumerate(model_scores_data):
    if "minimax" in model["name"].lower():
        continue 
    base_x = x_pos[i]
    color = family_colors_base.get(model['family'], '#000000') 
    
    # 1. 绘制连接线 (趋势线)
    x_coords_line = [base_x + X_JITTER[j] for j in range(3)]
    y_coords_line = model['scores']
    
    ax.plot(x_coords_line, y_coords_line, 
            color=color, 
            linestyle='-', 
            linewidth=2.5, 
            alpha=0.7, 
            zorder=1)

    # 2. 绘制数据点 (T0, T1, T2)
    for j in range(3):
        x = base_x + X_JITTER[j]
        y = model['scores'][j]
        
        marker_color = 'black'
        marker_size = 10 
        
        if j == 0: 
            marker = 'o' 
            alpha = 0.5
        elif j == 1: 
            marker = 'v'
            alpha = 0.8
        else: # j == 2 (T_k=2)
            marker = 's'
            alpha = 1.0
            # 收集 T2 点坐标
            t2_x_coords.append(x)
            t2_y_coords.append(y)
        
        ax.plot(x, y, 
                marker=marker, markersize=marker_size, 
                linestyle='None', 
                color=color, 
                markeredgecolor=marker_color, 
                markeredgewidth=0.8, 
                alpha=alpha, 
                zorder=2)
        
        if (j, marker) not in markers_drawn:
            legend_handles.append(ax.plot([], [], marker=marker, markersize=marker_size, 
                                          linestyle='None', color='gray', label=ELIMINATION_LABELS[j])[0])
            markers_drawn.add((j, marker))

# F. 绘制 T_k=2 的水平虚线 (新增部分)
ax.plot(t2_x_coords, t2_y_coords, 
        color='gray', 
        linestyle='--', # 虚线
        linewidth=1.5, 
        alpha=0.8, 
        zorder=0) # 放在最底层

# E. 图表配置和美化 (沿用 V9 设置)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_linewidth(1.5) 
ax.spines['left'].set_linewidth(1.5) 

ax.set_xticks(x_pos)
ax.set_xticklabels([model['simplified_name'] for model in model_scores_data], 
                   rotation=45, ha='right', fontsize=MODEL_LABEL_SIZE) 

ax.set_ylabel("Expected Win Score ($E[S_m]$)", fontsize=AXIS_LABEL_SIZE)
ax.grid(True, which='major', linestyle='--', linewidth=0.5, alpha=0.4, axis='y', zorder=0)
ax.set_xlabel("", fontsize=AXIS_LABEL_SIZE) 

# 使用用户提供的 Y 轴范围
ax.set_ylim(-0.1, 10.0)

# 7. 绘制图例 (Legend) - 沿用 V9 修正的位置

# 创建家族图例 (颜色)
family_handles = []
for family in sorted(families):
    color = family_colors_base.get(family)
    handle = ax.plot([], [], marker='s', markersize=10, linestyle='-', linewidth=2.5, color=color, alpha=0.8, 
                     markeredgecolor='black', markeredgewidth=0.8, label=f'{family} Family')[0]
    family_handles.append(handle)

# 家族图例：设置为 4 列 (两行), 位置在 1.08
legend1 = ax.legend(handles=family_handles, labels=[h.get_label() for h in family_handles], 
                    loc='upper center', bbox_to_anchor=(0.5, 1.05), # 调整到 1.08
                    ncol=4, 
                    frameon=False, fontsize=LEGEND_FONT_SIZE,
                    columnspacing=1.5, handletextpad=0.5)

ax.add_artist(legend1)

# 阶段图例：保持单行, 位置在 0.95
legend2 = ax.legend(handles=legend_handles, labels=[h.get_label() for h in legend_handles],
                    loc='upper center', bbox_to_anchor=(0.5, 0.9), # 调整到 0.95
                    ncol=len(legend_handles), 
                    frameon=False, fontsize=LEGEND_FONT_SIZE,
                    columnspacing=1.5, handletextpad=0.5)

# 调整 top 区域，使其包含图例 (从 0.80 调整到 0.90)
plt.subplots_adjust(bottom=0.20, top=0.90, left=0.05, right=0.98) 

plt.savefig("swiss_sensitivity_high.png", dpi=200, bbox_inches='tight')
plt.show()